# Title

Author: Sandra Godinho Silva \
Most updated version: 0.1 (1/01/2021)

## Purpose
State the purpose of the notebook.

## Methodology
Quickly describe assumptions and processing steps.

## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:
- todo 1;
- todo 2;
- todo 3.

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

import matplotlib as plt

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

## Local library import
We import all the required local libraries libraries

In [2]:
# Include local library paths
import sys
# sys.path.append('path/to/local/lib') # uncomment and fill to import local libraries

# Import local libraries

In [3]:
import numpy as np
from sklearn import model_selection

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import label_binarize


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold,  GridSearchCV



from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score, matthews_corrcoef, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.linear_model import LogisticRegression

import numpy as np

from sklearn.svm import SVC

from sklearn.multiclass import OneVsRestClassifier

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [4]:
num_runs = 1 #int(config.get('Evaluation', 'NumberRuns'))
num_test = 3
normalization = "Standard"

train_rf = "True"
train_logistic_regression = "False"
train_svm = "True"

# RF
NumberTrees = 500 
ValidationModels =  5

In [5]:
to_train = []

if train_rf == "True":
    to_train.append("RF")
if train_logistic_regression == "False":
    to_train.append("Logistic Regression")
if train_svm == "True":
    to_train.append("SVM")
    
# Set up DataFrames to store results
cv_list = ["Run_" + str(x) + "_CV_" + str(y) for x in range(num_runs) for y in range(num_test)]
auc_df = pd.DataFrame(index=to_train, columns=cv_list)
mcc_df = pd.DataFrame(index=to_train, columns=cv_list)
precision_df = pd.DataFrame(index=to_train, columns=cv_list)
recall_df = pd.DataFrame(index=to_train, columns=cv_list)
f1_df = pd.DataFrame(index=to_train, columns=cv_list)


# Data import
We retrieve all the required data for the analysis.

In [6]:
df = pd.read_csv("cog_genus_counts.csv", index_col=0)
df.head()

,COG1943,COG0756,COG1209,COG4942,COG1305,COG2307,COG2308,COG0726,COG2010,COG2133,COG2207,COG1082,COG0596,COG5485,COG2318,COG1733,COG1359,COG1472,COG2220,COG0366,COG1764,COG0583,COG3598,COG0484,COG1396,...,COG4034,COG4991,COG3115,COG3426,COG2039,COG3722,COG5036,COG4955,COG4133,COG2164,COG1513,COG3504,COG5405,COG1790,COG3356,COG3603,COG5324,COG3981,COG3431,COG3334,COG3635,COG3692,COG4691,COG4246,Genus
Assembly accession,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
GCA_000218485.2,3,1,2,1,3,1,2,1,3,3,14,0,8,0,1,3,0,3,0,0,1,4,0,3,1,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,Aquimarina
GCA_000255455.1,1,1,3,1,1,0,0,0,1,1,20,1,8,0,1,1,1,4,0,5,1,3,0,2,5,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Aquimarina
GCA_000430545.1,2,1,2,1,1,0,0,1,2,0,31,0,13,0,0,3,4,1,1,0,1,7,0,3,4,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Tenacibaculum
GCA_000430645.1,1,1,3,1,1,0,0,5,2,2,73,2,19,0,1,4,2,7,6,10,0,4,0,2,2,...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aquimarina
GCA_000430665.1,2,1,2,1,2,0,0,2,2,0,30,0,12,0,1,4,0,3,5,5,1,4,0,4,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aquimarina


# Data processing
Put here the core of the notebook. Feel free to further split this section into subsections.

In [7]:
df = df.copy()

In [8]:
category= "Genus"

In [9]:
labels = df.reset_index()[category]
labels_data, label_set = pd.factorize(labels.values) 
print(labels_data)
labels = labels_data
num_class = len(np.unique(labels_data))
print(num_class)

[0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0
 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
2


In [10]:
print(label_set)

['Aquimarina' 'Tenacibaculum']


In [11]:
features = df.index.values
full_features = df.index

In [12]:
# Get relative abundance per genome
df["sum"] = df.sum(axis=1)
numeric_cols = df.select_dtypes(exclude=["object"]).columns.to_list()
numeric_cols.remove("sum")
df[numeric_cols] = df[numeric_cols].div(df["sum"], axis=0).multiply(100)
df = df.reset_index().drop(columns=["sum",  "Assembly accession"])
df.head()

,COG1943,COG0756,COG1209,COG4942,COG1305,COG2307,COG2308,COG0726,COG2010,COG2133,COG2207,COG1082,COG0596,COG5485,COG2318,COG1733,COG1359,COG1472,COG2220,COG0366,COG1764,COG0583,COG3598,COG0484,COG1396,...,COG4034,COG4991,COG3115,COG3426,COG2039,COG3722,COG5036,COG4955,COG4133,COG2164,COG1513,COG3504,COG5405,COG1790,COG3356,COG3603,COG5324,COG3981,COG3431,COG3334,COG3635,COG3692,COG4691,COG4246,Genus
0,0.147710,0.049237,0.098474,0.049237,0.147710,0.049237,0.098474,0.049237,0.147710,0.147710,0.689316,0.000000,0.393895,0.0,0.049237,0.147710,0.000000,0.147710,0.000000,0.000000,0.049237,0.196947,0.0,0.147710,0.049237,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.049237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Aquimarina
1,0.048900,0.048900,0.146699,0.048900,0.048900,0.000000,0.000000,0.000000,0.048900,0.048900,0.977995,0.048900,0.391198,0.0,0.048900,0.048900,0.048900,0.195599,0.000000,0.244499,0.048900,0.146699,0.0,0.097800,0.244499,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0489,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Aquimarina
2,0.089526,0.044763,0.089526,0.044763,0.044763,0.000000,0.000000,0.044763,0.089526,0.000000,1.387645,0.000000,0.581916,0.0,0.000000,0.134288,0.179051,0.044763,0.044763,0.000000,0.044763,0.313339,0.0,0.134288,0.179051,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tenacibaculum
3,0.032520,0.032520,0.097561,0.032520,0.032520,0.000000,0.000000,0.162602,0.065041,0.065041,2.373984,0.065041,0.617886,0.0,0.032520,0.130081,0.065041,0.227642,0.195122,0.325203,0.000000,0.130081,0.0,0.065041,0.065041,...,0.03252,0.03252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Aquimarina
4,0.080257,0.040128,0.080257,0.040128,0.080257,0.000000,0.000000,0.080257,0.080257,0.000000,1.203852,0.000000,0.481541,0.0,0.040128,0.160514,0.000000,0.120385,0.200642,0.200642,0.040128,0.160514,0.0,0.160514,0.200642,...,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Aquimarina


In [13]:
y = df.pop(category).values

In [14]:
df = df.to_numpy()
df

array([[0.14771049, 0.04923683, 0.09847366, ..., 0.        , 0.        ,
        0.        ],
       [0.04889976, 0.04889976, 0.14669927, ..., 0.        , 0.        ,
        0.        ],
       [0.08952551, 0.04476276, 0.08952551, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.0567215 , 0.11344299, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05546312, 0.11092623, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05678592, 0.11357183, ..., 0.        , 0.        ,
        0.        ]])

In [15]:
# Set up seeds for different runs
rf_scores = pd.DataFrame(index=features)
        
seeds = np.random.randint(1000, size=num_runs)
run = 0

In [16]:
category="Genus"
#X = df.drop(‘class’, axis=1)
#y = df[category]

X = df.copy()
y = labels.copy()

In [17]:
#https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652

In [18]:
# Hyperparameter tunning
def parameter_tunning_RF():
    # number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    
    # number of features at every split
    max_features = ["auto", "sqrt"]

    # max depth
    max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
    max_depth.append(None)
    
    # create random grid
    random_grid = {
     "n_estimators": n_estimators,
     "max_features": max_features,
     "max_depth": max_depth
     }
    
    rfc = RandomForestClassifier()
    # Random search of parameters
    rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    
    # Fit the model
    rfc_random.fit(X_train, y_train)
       
    best_paramters = rfc_random.best_params_
    print(best_paramters)
    
    return best_paramters

In [19]:
def get_stats(y_test, rfc_predict):
    stat_dict = {}
    # View accuracy score

    stat_dict["Accuracy"] = accuracy_score(y_test, rfc_predict)
    #stat_dict["Confusion_matrix"] = confusion_matrix(y_test, rfc_predict)
    #stat_dict["Classification_report"] = classification_report(y_test, rfc_predict)
    #stat_dict["AUC Scores"] = rfc_cv_score
    stat_dict["Mean AUC Score"] = rfc_cv_score.mean()
    
    stat_dict["MCC"] = matthews_corrcoef(y_test, rfc_predict)
    stat_dict["Precision"] = precision_score(y_test, rfc_predict, average='weighted')
    stat_dict["Recall"] = recall_score(y_test, rfc_predict, average='weighted')
    stat_dict["F1"] = f1_score(y_test, rfc_predict, average='weighted')

    #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores
    if num_class == 2:
        stat_dict["AUC"] = roc_auc_score(y_test, rfc.predict_proba(X_test)[:,1], average='weighted')
    else:
        stat_dict["AUC"] = roc_auc_score(y_test, rfc.predict_proba(X_test), average='weighted')   
    
    print(stat_dict)
    
    return stat_dict

In [20]:
# Set up seeds for different runs
if num_class == 2:
    svm_scores = pd.DataFrame(index=y)
    logistic_regression_scores = pd.DataFrame(index=y)
else:
    svm_scores = {}
    logistic_regression_scores = {}
    for l in label_set:
        svm_scores[l] = pd.DataFrame(index=y)     
        logistic_regression_scores[l] = pd.DataFrame(index=y) 

In [21]:
normalization="Standard"
for seed in seeds:
    print("Starting CV")
    skf = StratifiedKFold(n_splits=num_test, shuffle=True, random_state=seed)
    fold = 0
    
    # Hypertunning before K folds - Random Forest
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)
    best_parameters = parameter_tunning_RF()
    
    # Stratified k-fold
    for train_index, test_index in skf.split(X, y):
        # Select and format training and testing sets
        X_train, X_test = df[train_index,:], df[test_index,:]
        y_train, y_test = labels[train_index], labels[test_index]
        
        cl =  np.unique(y_train)
        num_class = len(cl)
        
        # Data pre-processing with StandardScaler or MinMaxScaler.
        if normalization == "MinMax":
            scaler = MinMaxScaler().fit(X_train)
            X_train = np.clip(scaler.transform(X_train), 0, 1)
            X_test = np.clip(scaler.transform(X_test), 0, 1)

        if normalization == "Standard":
            scaler = StandardScaler().fit(X_train)
            X_train = np.clip(scaler.transform(X_train), -3, 3)
            X_test = np.clip(scaler.transform(X_test), -3, 3)
        
        # RANDOM FOREST
        if train_rf == "True":
        # Already use best parameters from Hyperparameter tunning
            rfc = RandomForestClassifier(n_estimators=best_parameters["n_estimators"], max_features=best_parameters["max_features"], max_depth=best_parameters["max_depth"])

            # Fit model
            rfc.fit(X_train,y_train)
        
            # Get predicted labels for the test data:
            rfc_predict = rfc.predict(X_test)

            # Evaluate Performance with cross-validation 
            rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
        
            # Get ranking of feature importance
            feature_importance = rfc.feature_importances_
        
            # Get results
            stat_dict = get_stats(y_test, rfc_predict)
        
            auc_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stat_dict["AUC"]
            mcc_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stat_dict["MCC"]
            precision_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stat_dict["Precision"]
            recall_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stat_dict["Recall"]
            f1_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stat_dict["F1"]

            sys.stdout.write("")
            print("# RF:\t\t\t%f\t%f" % (stat_dict["AUC"], auc_df.loc["RF"].mean(axis=0)))
        
        fold +=1
    run += 1

Starting CV
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   50.7s finished


{'n_estimators': 1000, 'max_features': 'auto', 'max_depth': 140}
{'Accuracy': 0.8888888888888888, 'Mean AUC Score': 1.0, 'MCC': 0.7567593452331792, 'Precision': 0.8905050505050505, 'Recall': 0.8888888888888888, 'F1': 0.886574074074074, 'AUC': 0.9899665551839465}
# RF:			0.989967	0.989967
{'Accuracy': 1.0, 'Mean AUC Score': 1.0, 'MCC': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0, 'AUC': 1.0}
# RF:			1.000000	0.994983
{'Accuracy': 1.0, 'Mean AUC Score': 1.0, 'MCC': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0, 'AUC': 1.0}
# RF:			1.000000	0.996656


In [22]:
auc_df

,Run_0_CV_0,Run_0_CV_1,Run_0_CV_2
RF,0.989967,1,1
SVM,NaN,NaN,NaN


In [26]:
# Third-party libraries

gaussian=False
seed=42

#SVM
GridCV=2
MaxIterations=5
run=0
if train_svm == "True":
    print(run)
    num_cv = GridCV
    max_iter = MaxIterations  
    
    if num_class > 2:
        train_y_binarize = label_binarize(y_train, classes=cl)
        test_y_binarize = label_binarize(y_test, classes=cl)
        
        if gaussian==True:
            grid = [{'estimator__kernel': ['rbf'], 'estimator__gamma': [1e-3, 1e-4], 'estimator__C': [1, 10, 100, 1000]},
                          {'estimator__kernel': ['linear'], 'estimator__C': [1, 10, 100, 1000]}]
        else:
            grid = [{'estimator__C': [1, 10, 100, 1000], 'estimator__kernel': ['linear']}]
            
        clf = GridSearchCV(OneVsRestClassifier(SVC(probability=True, max_iter=max_iter)),
                               grid, cv=StratifiedKFold(num_cv).split(X_train, y_train), 
                               scoring="roc_auc", n_jobs=-1)
        
        clf.fit(train_x, train_y_binarize)

    else:
        if gaussian==True:
            grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
                          {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}] 
        else:
            grid = [ {'C': [1, 10, 100, 1000], 'kernel': ['linear']}]

        clf = GridSearchCV(SVC(probability=True, max_iter=max_iter), 
                               grid, cv=StratifiedKFold(num_cv).split(X_train, y_train), scoring="roc_auc", n_jobs=-1)
        clf.fit(X_train, y_train)

    test_probs = clf.predict_proba(X_test)
    svm_pred = np.argmax(clf.predict_proba(X_test), axis=1)

    test_stat_dict = get_stats(y_test, svm_pred)
    
    if num_class == 2 and gaussian == False:
        weights = np.array(clf.best_estimator_.coef_).reshape(-1)
    elif num_class > 2 and gaussian == False:
        weights = np.array(clf.best_estimator_.coef_)
    elif gaussian == True:
        weights = None

    if num_class == 2:
        svm_scores["Run_" + str(run) + "_CV_" + str(fold)] = weights
    else:
        for l in range(len(label_set)):
            score_list = weights[l,:].reshape(-1)
            lab = label_set[l]
            svm_scores[lab]["Run_" + str(run) + "_CV_" + str(fold)] = score_list
                            
            auc_df.loc["SVM"]["Run_" + str(run) + "_CV_" + str(fold)]=test_stat_dict["AUC"]
            mcc_df.loc["SVM"]["Run_" + str(run) + "_CV_" + str(fold)]=test_stat_dict["MCC"]
            precision_df.loc["SVM"]["Run_" + str(run) + "_CV_" + str(fold)]=test_stat_dict["Precision"]
            recall_df.loc["SVM"]["Run_" + str(run) + "_CV_" + str(fold)]=test_stat_dict["Recall"]
            f1_df.loc["SVM"]["Run_" + str(run) + "_CV_" + str(fold)]=test_stat_dict["F1"]

            sys.stdout.write("")
            print("# SVM:\t\t\t%f\t%f" % (test_stat_dict["AUC"], auc_df.loc["SVM"].mean(axis=0)))

        fold +=1
    run += 1
    print("add_run")

0
{'Accuracy': 1.0, 'Mean AUC Score': 1.0, 'MCC': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0, 'AUC': 1.0}


/home/sandra/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=5).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


ValueError: Length of values (2321) does not match length of index (107)

In [ ]:
clf

In [ ]:
                    # Triain SVM model using raw and tree features
                    #################################################################

                
                   



In [ ]:
clf

In [ ]:
y

In [ ]:
weights

In [ ]:
def train_logistic_regression(train, test, config, seed=42, regularization = True):

    
            

    return clf, test_stat_dict, weights

# Other trials

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve
import matplotlib.pyplot as plt

for seed in seeds:
    print("Starting CV")
    skf = StratifiedKFold(n_splits=num_test, shuffle=True, random_state=seed)
    fold = 0
    for train_index, test_index in skf.split(df, labels):
        stat_dict = {}
        
        # Select and format training and testing sets
        X_train, X_test = df[train_index,:], df[test_index,:]
        y_train, y_test = labels[train_index], labels[test_index]
        print(X_train.shape, X_test.shape, df.shape)
        
        RandomForestClassifier(n_estimators=1000, max_depth=5, class_weight='balanced')
        clf.fit(X_train,y_train)
        

        

In [ ]:
y_proba

In [ ]:
for seed in seeds:
    print("Starting CV")
    skf = StratifiedKFold(n_splits=num_test, shuffle=True, random_state=seed)
    fold = 0
    for train_index, test_index in skf.split(df, labels):
        
        # Select and format training and testing sets
        train_x, test_x = df[train_index,:], df[test_index,:]
        train_y, test_y = labels[train_index], labels[test_index]
        
        num_train_samples = train_x.shape[0]
        num_test_samples = test_x.shape[0]
                
        if normalization == "MinMax":
            scaler = MinMaxScaler().fit(train_x)
            train_x = np.clip(scaler.transform(train_x), 0, 1)
            test_x = np.clip(scaler.transform(test_x), 0, 1)

        if normalization == "Standard":
            scaler = StandardScaler().fit(train_x)
            train_x = np.clip(scaler.transform(train_x), -3, 3)
            test_x = np.clip(scaler.transform(test_x), -3, 3)
        
        train_x = np.array(train_x)
        train_y = np.array(train_y) 
        
        train = [train_x, train_y]
        test = [test_x, test_y]
        
        
        # Triain RF model using raw and tree features
        if train_rf == "True":

            _, stats, scores = train_rf(train, test)
            print("done")
            rf_scores["Run_" + str(run) + "_CV_" + str(fold)] = scores

            auc_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["AUC"]
            mcc_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["MCC"]
            precision_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["Precision"]
            recall_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["Recall"]
            f1_df.loc["RF"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["F1"]

            sys.stdout.write("")
            print("# RF:\t\t\t%f\t%f" % (stats["AUC"], auc_df.loc["RF"].mean(axis=0)))

In [ ]:
#np.eye(len(np.unique(y)))[np.array(y).reshape(-1)]

In [ ]:
    number_trees = NumberTrees
    num_models = ValidationModels

    x, y = train
    test_x, test_y = test
    
    clf = RandomForestClassifier(n_estimators=number_trees, n_jobs=-1)
    clf.fit(x, y)
    
    feature_importance = clf.feature_importances_

    test_probs = np.array([row for row in clf.predict_proba(test_x)])
    test_pred = np.argmax(test_probs, axis=-1)
    probs = np.array(test_probs)
    
    
    stat_dict = {}
    
    y = np.array(y)
    y_oh = np.eye(len(np.unique(y)))[np.array(y).reshape(-1)]
    #Binary case:
    if num_class == 2:
        stat_dict["AUC"] = roc_auc_score(y, clf.predict_proba(test_x)[:, 1], average='weighted')
    #Multiclass case:
    else:
        stat_dict["AUC"] = roc_auc_score(y_oh, probs, average='weighted')     

    #return clf, test_stat_dict, feature_importance

In [ ]:
probs = test_probs

y = np.array(y)
y_oh = np.eye(len(np.unique(y)))[np.array(y).reshape(-1)]
probs = np.array(probs)

num_class = probs.shape[1]
stat_dict = {}
         
if num_class == 2:
    stat_dict["AUC"] = roc_auc_score(y, probs[:,1], average='weighted')
else:
    stat_dict["AUC"] = roc_auc_score(y_oh, probs, average='weighted')        
stat_dict["MCC"] = matthews_corrcoef(y, pred.reshape(-1))
stat_dict["Precision"] = precision_score(y, pred, average='weighted')
stat_dict["Recall"] = recall_score(y, pred, average='weighted')
stat_dict["F1"] = f1_score(y, pred, average='weighted')

In [ ]:
print(clf)

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2